![Photo by Stephen Phillips - Hostreviews.co.uk on UnSplash](https://cf.bstatic.com/xdata/images/hotel/max1024x768/408003083.jpg?k=c49b5c4a2346b3ab002b9d1b22dbfb596cee523b53abef2550d0c92d0faf2d8b&o=&hp=1){fig-align="center" width=50%}


# Import data

In [1]:
import time
import warnings
from pathlib import Path

import catboost
import numpy as np
import pandas as pd
from data import pre_process, utils
from lets_plot import *
from lets_plot.mapping import as_discrete
from sklearn import compose, impute, model_selection, pipeline, preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

LetsPlot.setup_html()

**Objective**:  
* jh
.





# Prepare dataframe before modelling


In [2]:
X, y = pre_process.prepare_data_for_modelling()

# https://www.kdnuggets.com/2023/02/top-5-advantages-catboost-ml-brings-data-make-purr.html
for col in X.columns:
    if X[col].dtype.name in ("bool", "object", "category"):
        X[col] = X[col].fillna("missing value")

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=utils.Configuration.seed
)

print(f"Shape of X-train: {X_train.shape}")
print(f"Shape of X-test: {X_test.shape}")

Shape of X and y: (3660, 49), (3660,)
Shape of X-train: (2928, 49)
Shape of X-test: (732, 49)


## Selecting categorical and continous variables

In [3]:
numerical_features = X_train.select_dtypes("number").columns.to_list()
categorical_features = X_train.select_dtypes("object").columns.to_list()

assert len(numerical_features) + len(categorical_features) == X_train.shape[1]

In [4]:
train_dataset = catboost.Pool(X_train, y_train, cat_features=categorical_features)
test_dataset = catboost.Pool(X_test, y_test, cat_features=categorical_features)